In [1]:
import copy
import numpy as np

from numpy.random import shuffle
import matplotlib.pyplot as plt
from tqdm import tqdm

import funcs as f
import utils as utl

In [2]:
def show(G):
    
    tmp = 'X | '
    for v in V:
        tmp += ' ' + v + ' '
    
    print(tmp)
    print('')
    
    for row in range(len(V)):
        
        tmp = V[row] + ' | '
        for col in range(Vsize):
            
            if G[row][col] == np.inf:
                tmp +=  ' i '
            else:
                w = int(G[row][col])
                if w == 0:
                    tmp +=  " - "
                else:
                    if w < 10 : tmp +=  ' ' + str(w) + ' '
                    if 10 <= w < 100 : tmp +=  ' ' + str(w)

            
        print(tmp)

![title](test_one.jpg)

In [3]:
# |V| = |{S,E}| + |T|*|K| = 2 + 4*2 = 10
Vsize = 6
Esize = 9

#     0    1    2    3    4    5 
V = ['S', 'a', 'b', 'c', 'd', 'E']

In [116]:
g = np.zeros((Vsize,Vsize), dtype=np.float32)
# S
g[0][1] = np.inf 

# a
g[1][2] = 5  #q_t(k)

# b
g[2][3] = 20
g[2][1] = np.inf


#c
g[3][4] = 5  #q_t(k)
g[3][2] = np.inf

#d
g[4][5] = 90  #q_t(k)
g[4][3] = np.inf

In [117]:
show(g)

X |  S  a  b  c  d  E 

S |  -  i  -  -  -  - 
a |  -  -  5  -  -  - 
b |  -  i  -  20 -  - 
c |  -  -  i  -  5  - 
d |  -  -  -  i  -  90
E |  -  -  -  -  -  - 


In [118]:
g[g > 0].size # Esize

8

---

In [119]:
#@njit
def iteration(G, F, V, _inf=np.inf):
    
    Prev = np.full(Vsize, -1) # Prev node
    Q = [0] # Queue
    cursor = 0 # instead of popping elements of of the Q

    while cursor < Vsize:

        v_curr = Q[cursor]
        for v_ind, capacity in enumerate( G[v_curr] ):
            if F[v_curr, v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
                Q.append(v_ind)
                Prev[v_ind] = v_curr

        cursor += 1
        if cursor >= len(Q) : break

    # path reconstruction
    v_curr = Vsize - 1
    path = [v_curr]
    while True:
        v_prev = Prev[v_curr]
        v_curr = v_prev
        if v_curr == -1 : break
        path.append(v_curr)

    path = path[::-1]
    print(path)
    
    
    # path bottleneck
    bottleneck = _inf # float('inf') doesn't work with numba
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        if G[v_curr, v_next] < bottleneck: 
            bottleneck = G[v_curr, v_next]
    if bottleneck == _inf : return F, True
    
    
    # path flow update
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        F[v_curr, v_next] += bottleneck
        
    
    #print('')
    #print('----- New iteration -----')
    #show_path(path, V)
    #print('Bottleneck is: ', bottleneck)
    
    return F, False

In [120]:
Vsize = g.shape[0]
F_ = np.zeros((Vsize, Vsize)) # Current flow

In [121]:
while True:
        F_, end = iteration(g, F_, V)
        if end : break
            
print('MaxFlow is: ', sum(F_[:, -1]))

[0, 1, 2, 3, 4, 5]
[5]
MaxFlow is:  5.0


In [122]:
show(g)

X |  S  a  b  c  d  E 

S |  -  i  -  -  -  - 
a |  -  -  5  -  -  - 
b |  -  i  -  20 -  - 
c |  -  -  i  -  5  - 
d |  -  -  -  i  -  90
E |  -  -  -  -  -  - 


In [123]:
show(F_)

X |  S  a  b  c  d  E 

S |  -  5  -  -  -  - 
a |  -  -  5  -  -  - 
b |  -  -  -  5  -  - 
c |  -  -  -  -  5  - 
d |  -  -  -  -  -  5 
E |  -  -  -  -  -  - 


## MinCut

In [124]:
# BFS
Prev = np.full(Vsize, -1) # Prev node
Q = [0] # Queue
cursor = 0 # instead of popping elements of of the Q

while True:

    v_curr = Q[cursor]
    for v_ind, capacity in enumerate( g[v_curr] ):
        if 0 < F_[v_curr][v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
            Q.append(v_ind)
            Prev[v_ind] = v_curr
    
    cursor += 1
    if cursor >= len(Q) : break

In [125]:
# S a b c d E
# 0 1 2 3 4 5
Q

[0, 1]